## Hierarchical Indexing
Eber David Gaytan Medina

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas Series and DataFrame objects, respectively. Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys. While Pandas does provide Panel and Panel4D objects that natively handle three-dimensional and four-dimensional data (see Aside: Panel Data), a far more common pattern in practice is to make use of hierarchical indexing (also known as multi-indexing) to incorporate multiple index levels within a single index. In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.


In [ ]:
import pandas as pd
import numpy as np


index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop
(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

pop[('California', 2010):('Texas', 2000)]
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

pop[[i for i in pop.index if i[1] == 2010]]
(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

index = pd.MultiIndex.from_tuples(index)
index
MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

pop = pop.reindex(index)
pop
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

pop[:, 2010]
California    37253956
New York      19378102
Texas         25145561
dtype: int64

pop_df = pop.unstack()
pop_df
2000	2010
California	33871648	37253956
New York	18976457	19378102
Texas	20851820	25145561

pop_df.stack()
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df
total	under18
California	2000	33871648	9267089
2010	37253956	9284094
New York	2000	18976457	4687374
2010	19378102	4318033
Texas	2000	20851820	5906301
2010	25145561	6879014

f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()
2000	2010
California	0.273594	0.249211
New York	0.247010	0.222831
Texas	0.283251	0.273568

df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df
data1	data2
a	1	0.554233	0.356072
2	0.925244	0.219474
b	1	0.441759	0.610054
2	0.171495	0.886688


pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])
MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)]
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])
MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])
MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data
subject	Bob	Guido	Sue
type	HR	Temp	HR	Temp	HR	Temp
year	visit						
2013	1	31.0	38.7	32.0	36.7	35.0	37.2
2	44.0	37.7	50.0	35.0	29.0	36.7
2014	1	30.0	37.4	39.0	37.8	61.0	36.9
2	47.0	37.8	48.0	37.3	51.0	36.5

health_data['Guido']
type	HR	Temp
year	visit		
2013	1	32.0	36.7
2	50.0	35.0
2014	1	39.0	37.8
2	48.0	37.3

pop
state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

pop['California', 2000]
33871648

pop['California']
year
2000    33871648
2010    3725395
pop.loc['California':'New York']
state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

pop[:, 2000]
state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

pop[pop > 22000000]
state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

pop[['California', 'Texas']]
state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

health_data
subject	Bob	Guido	Sue
type	HR	Temp	HR	Temp	HR	Temp
year	visit						
2013	1	31.0	38.7	32.0	36.7	35.0	37.2
2	44.0	37.7	50.0	35.0	29.0	36.7
2014	1	30.0	37.4	39.0	37.8	61.0	36.9
2	47.0	37.8	48.0	37.3	51.0	36.5
health_data['Guido', 'HR']
year  visit
2013  1        32.0
      2        50.0
2014  1        39.0
      
health_data.iloc[:2, :2]
subject	Bob
type	HR	Temp
year	visit		
2013	1	31.0	38.7
2	44.0	37.7

health_data.loc[:, ('Bob', 'HR')]
year  visit
2013  1        31.0
      2        44.0
2014  1        30.0
      2        47.0
Name: (Bob, HR), dtype: float64

health_data.loc[(:, 1), (:, 'HR')]
  File "<ipython-input-32-8e3cc151e316>", line 1
    health_data.loc[(:, 1), (:, 'HR')]

idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]
subject	Bob	Guido	Sue
type	HR	HR	HR
year	visit			
2013	1	31.0	32.0	35.0
2014	1	30.0	39.0	61.0

index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data
char  int
a     1      0.003001
      2      0.164974
c     1      0.741650
      2      0.569264
b     1      0.001693
      2      0.526226
dtype: float64

try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)
<class 'KeyError'>

data = data.sort_index()
data
char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
c     1      0.741650
      2      0.569264
dtype: float64

data['a':'b']
char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
dtype: float64

pop.unstack(level=0)
state	California	New York	Texas
year			
2000	33871648	18976457	20851820
2010	37253956	19378102	25145561
pop.unstack(level=1)
year	2000	2010
state		
California	33871648	37253956
New York	18976457	19378102
Texas	20851820	25145561

pop.unstack().stack()
state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561

pop_flat = pop.reset_index(name='population')
pop_flat
state	year	population
0	California	2000	33871648
1	California	2010	37253956
2	New York	2000	18976457
3	New York	2010	19378102
4	Texas	2000	20851820
5	Texas	2010	25145561
pop_flat.set_index(['state', 'year'])
population
state	year	
California	2000	33871648
2010	37253956
New York	2000	18976457
2010	19378102
Texas	2000	20851820
2010	25145561

type	HR	Temp	HR	Temp	HR	Temp
year	visit						
2013	1	31.0	38.7	32.0	36.7	35.0	37.2
2	44.0	37.7	50.0	35.0	29.0	36.7
2014	1	30.0	37.4	39.0	37.8	61.0	36.9
2	47.0	37.8	48.0	37.3	51.0	36.5

data_mean = health_data.mean(level='year')
data_mean
subject	Bob	Guido	Sue
type	HR	Temp	HR	Temp	HR	Temp
year						
2013	37.5	38.2	41.0	35.85	32.0	36.95
2014	38.5	37.6	43.5	37.55	56.0	36.70

data_mean.mean(axis=1, level='type')
type	HR	Temp
year		
2013	36.833333	37.000000
2014	46.000000	37.283333
